<img src="https://prnewswire2-a.akamaihd.net/p/1893751/sp/189375100/thumbnail/entry_id/1_su9da4fu/def_height/1001/def_width/1911/version/100011/type/2/q/100"  width="300" height="200">


Who is Zillow?
> "Zillow Group, Inc. operates real estate brands on mobile and the web in the United States. It operates through three segments: Homes; Internet, Media & Technology; and Mortgages. The company's platform offers buying, selling, renting, and financing services for residential real estate. It also provides a suite of marketing software and technology solutions; and advertising services." [source](https://finance.yahoo.com/quote/Z/profile?p=Z)
   
Why is this important to Zillow?
> Zillow's Zestimate is used as to estimate the current market value of a home. The Zestimate is calculated using public tax and property data. <strong>Zillow's business model is to provide real estate services</strong>!

The potential impact of Zillow Zestimate is:
> - Mortgage lenders take on less risk when they use Zillow's Zestimate. It's another metric a lender can take into consideration during the underwriting process. (Lenders will send their own agents/personnel to review the physical property)
> - Home buyers/sellers understand the estimated current market value of a property.

---
# Data Acquisition

Now that we've got our WHY, we'll need to acquire data to meet our client's request(s):

>Use the 2017 properties and predictions data for <strong>single unit / single family homes</strong>... [include] the discoveries you made and work you have done related to <strong> uncovering the drivers Zillow's Zestimate error.
    
Let's clearly define:<br>
    1. A single unit / single family home<br>
    2. Zillow's Zestimate error

<br>    
    
## What exactly is a single unit/ single family home?
    
This definition comes from the [US Census](https://www.census.gov/construction/chars/definitions/) website.
    
<strong>Attached and Detached Single-Family Housing Units</strong>
>
> Single-family structures include fully detached, semi-detached (semi-attached, side-by-side), row houses, duplexes, quadruplexes, and townhouses. In order for attached units to be classified as single-family structures, each unit must:
>
> - Be separated by a ground-to roof wall,
> - Have a separate heating system,
> - Have individual meters for public utilities, and
> - Have no units located above or below.
>
> If each unit within the building does not meet the conditions above, the building is considered multifamily.


Another definition from The City of Los Angeles Municipal [codes](https://codelibrary.amlegal.com/codes/los_angeles/latest/lapz/0-0-0-886).
    
    
<strong>DWELLING UNIT</strong>
>     
> A group of two or more rooms, one of which is a kitchen, designed for occupancy by one family for living and sleeping purposes.  (Amended by Ord. No. 107,884, Eff. 9/23/56.)    
    
<strong>FAMILY</strong>
>
> One or more persons living together in a dwelling unit, with common access to, and common use of all living, kitchen, and eating areas within the dwelling unit.  (Amended by Ord. No. 177,325, Eff. 3/18/06.)

<br>    
    
## What's a Zestimate? What does the Zestimate error represent? 

<strong>Zestimate</strong>
    
>The Zestimate® home valuation model is Zillow’s estimate of a home's market value. The Zestimate incorporates public and user-submitted data, taking into account home facts, location and market conditions.
    
<strong>Zestimate error</strong>

The Zestimate's accuracy is computed by comparing the final sale price to the Zestimate that was published on or just prior to the sale date.
>
>Median Error: Zillow calulcates the error between what a home sells for and their prediction as the error. Zillow then takes the median of these errors for a given city. [source](https://www.zillow.com/zestimate/)

---
Now we understand the potential impact our project has to improve Zillow's Zestimate, and help people who rely on their services. Let's create a map for the acquisition procecss so we know where we're going. 
    
# Data Acquisition Workflow
Step 1: SQL Query<br>
Step 2: Store and load local cached file


# Step 1: SQL Query
---
1. Database structure
1. Query
1. Load data from the database to local environment

## 1. Database Structure
Before we import the data to our local environment, we need to ask ourselves 3 _important_ questions.

1. What data do I need? / What data does my team need?

> I need data describing properties and predictions for single unit homes sold in 2017. I need this data to discover the drivers of logerror. It's crucial to know this step. Reminder to self:
> 
> `"What's the business problem I'm trying to solve for my client."`
>
> Acquiring irrelevant data wastes time and reveals irrelevant insights.

2. Where is the data stored and what form(s) is it in?


> Our Zillow data is stored on a SQL database under the database name: `zillow`. Relevant data to the project is spread across several tables (listed below).

3. Before I begin aquiring the data from the database, can I explain why I selected these tables?

> In addition to `properties_2017` and `prediction_2017`, several `{key: value}` tables are needed in our query. These table will act like a rosetta stone for our main table, `properties_2017`. In `properties_2017`, categorical values are stored as numeric values. We have no clue what the numbers represent without context. The `{key: value}` tables describe what each unique numeric value represents. <strong>We can use the actual `{key: value} names` during exploration to discover the drivers of Zillow's Zestimate logerror</strong>.

<br>

SQL Database Name: `zillow`

| SQL Table Name | Contents | Features |
| :------------- | :------- | :------- |
| `airconditioningtype` | Indicates the properties air conditioning type | `airconditioningtypeid`, `airconditioningdesc`|
| `architecturalstyletype` | Indicates the architectural style of the property | `architecturalstyletypeid`, `architecturalstyletype` |
| `buildingclasstype` | Indicates the type of fireproofing a property has | `buildingclasstypeid`, `buildingclassdesc` |
| `heatingorsystemtype` | Indicates how the property is heated | `heatingorsystemtypeid`, `heatingorsystemdesc` |
| `predictions_2017` | The last date a property was sold with a corresponding Zestimate logerror for 2017 | `parcelid`, `logerror`, `transactiondate` |
| `propertylandusetype` | The type of property that sits on a lot | `propertylandusetypeid`, `propertylandusedesc` |
| `storytype` | Indicates the number of stories a property has | `storytypeid`, `storydesc` |
| `typeconstructiontype` | Indicates the type of material used to construct the property | `typeconstructiontypeid`, `typeconstructiondesc` |
| `properties_2017` | Properties that were sold in 2017. Each property has unique id with features describing the property | `parcelid`,`id`,`basementsqft`,`bathroomcnt`, `bedroomcnt`,`buildingqualitytypeid`,`calculatedbathnbr`, `decktypeid`,`finishedfloor1squarefeet`,`calculatedfinishedsquarefeet`, `finishedsquarefeet12`,`finishedsquarefeet13`,`finishedsquarefeet15`, `finishedsquarefeet50`,`finishedsquarefeet6`, `fips`,`fireplacecnt`,`fullbathcnt`,`garagecarcnt`, `garagetotalsqft`,`hashottuborspa`,`latitude`,`longitude`, `lotsizesquarefeet`,`poolcnt`,`poolsizesum`,`pooltypeid10`, `pooltypeid2`,`pooltypeid7`,`propertycountylandusecode`, `propertyzoningdesc`,`rawcensustractandblock`, `regionidcity`,`regionidcounty`,`regionidneighborhood`, `regionidzip`,`roomcnt`,`threequarterbathnbr`,`unitcnt`, `yardbuildingsqft17`,`yardbuildingsqft26`,`yearbuilt`, `numberofstories`,`fireplaceflag`,`structuretaxvaluedollarcnt`, `taxvaluedollarcnt`,`assessmentyear`, `landtaxvaluedollarcnt`,`taxamount`,`taxdelinquencyflag`, `taxdelinquencyyear`,`censustractandblock`|

<br>
<br>

## 2. Query
Let's take a look at our main table: `properties_2017`. Using <strong><font color='blue'>select * from properties_2017;</font></strong>, I noticed many columns contained missing values.

![test](initial_query.png)


Because of this, tables that are joined onto `properties_2017` need to be joined using: <strong><font color='blue'> left join </font></strong>

<img src="https://images.squarespace-cdn.com/content/v1/5732253c8a65e244fd589e4c/1464122797709-C2CDMVSK7P4V0FNNX60B/ke17ZwdGBToddI8pDm48kMjn7pTzw5xRQ4HUMBCurC5Zw-zPPgdn4jUwVcJE1ZvWEV3Z0iVQKU6nVSfbxuXl2c1HrCktJw7NiLqI-m1RSK4p2ryTI0HqTOaaUohrI8PIO5TUUNB3eG_Kh3ocGD53-KZS67ndDu8zKC7HnauYqqk/image-asset.png"  width="600" height="500">

If the left table has a missing value at a certain observation, a left join preserves the original shape/columns/rows of the left table by inserting columns from the right table as a <strong><font color='blue'>NULL</font></strong> value.

> We want as much information from our data source that:
> 1. Is within the scope of the project/ relevant to the business problem.
> 2. Helps us to explain insights/ drivers of Zillow's Zestimate logerror.

Now that we know which tables to query and how to combine them, let's write the SQL required to retreive the data.

<br>

``` sql 
    select *
    from properties_2017
    join(select parcelid,
        logerror,
        max(transactiondate) as lasttransactiondate
        from predictions_2017
        group by parcelid, logerror
        ) as predictions using(parcelid)
    left join `airconditioningtype` using(`airconditioningtypeid`)
    left join `architecturalstyletype` using(`architecturalstyletypeid`)
    left join `buildingclasstype` using(`buildingclasstypeid`)
    left join `heatingorsystemtype` using(`heatingorsystemtypeid`)
    left join `propertylandusetype` using(`propertylandusetypeid`)
    left join `storytype` using(`storytypeid`)
    left join `typeconstructiontype` using(`typeconstructiontypeid`)
    where (latitude is not null
    and longitude is not null
    and propertylandusetypeid = 261);
```
You can find a break down of the SQL query in the [appendix](#Appendix).

# 3. Load data from the database to our local environment

Let's fetch the data!

1. Acquire the data using pandas - pd.read_sql()
1. Create a function to reproduce acquisition.
1. Store the data locally

## 1. Acquire the data
---
1. Import libraries
2. Create a get_connection function to access SQL database.
3. Create a variable to store the SQL query to acquire single unit / single family properties.

### Import libraries

In [8]:
# import libraries and helper functions
import numpy as np
import pandas as pd
# import database credentials
import env

from warnings import filterwarnings
filterwarnings('ignore')

### Create a get_connection function to access SQL database

In [9]:
# Create a function to format a string with credentials
def get_connection(db, user=env.user, host=env.host, password=env.password):
    '''
    Returns a formatted url string with login credentials to access a SQL database.
    '''
    return f'mysql+pymysql://{user}:{password}@{host}/{db}'

### Create a variable to store the SQL query to acquire single unit / single family properties
- Single family residential properties are assigned the id 261.

In [10]:
# Store the sql query covered above as a string and assign it to a variable.
sql_query ='''
    select *
    from properties_2017
    join(select parcelid,
        logerror,
        max(transactiondate) as lasttransactiondate
        from predictions_2017
        group by parcelid, logerror
        ) as predictions using(parcelid)
    left join `airconditioningtype` using(`airconditioningtypeid`)
    left join `architecturalstyletype` using(`architecturalstyletypeid`)
    left join `buildingclasstype` using(`buildingclasstypeid`)
    left join `heatingorsystemtype` using(`heatingorsystemtypeid`)
    left join `propertylandusetype` using(`propertylandusetypeid`)
    left join `storytype` using(`storytypeid`)
    left join `typeconstructiontype` using(`typeconstructiontypeid`)
    where (latitude is not null
    and longitude is not null
    and propertylandusetypeid = 261);
    '''

In [11]:
# Use pd.read_sql() to acquire data from a SQL database
# pass sql_query and get_connection('zillow') as arguements.
df = pd.read_sql(sql_query, get_connection('zillow'))

In [12]:
# Display the shape of our dataframe.
df.shape

(52439, 68)

In [13]:
# Display the first 5 rows of the data
df.head()

,typeconstructiontypeid,storytypeid,propertylandusetypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,parcelid,id,basementsqft,...,censustractandblock,logerror,lasttransactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
0,NaN,NaN,261.0,2.0,None,NaN,NaN,10711855,1087254,NaN,...,6.037113e+13,-0.007357,2017-07-07,None,None,None,Central,Single Family Residential,None,None
1,NaN,NaN,261.0,2.0,None,NaN,1.0,10711877,1072280,NaN,...,6.037113e+13,0.021066,2017-08-29,Central,None,None,Central,Single Family Residential,None,None
2,NaN,NaN,261.0,2.0,None,NaN,1.0,10711888,1340933,NaN,...,6.037113e+13,0.077174,2017-04-04,Central,None,None,Central,Single Family Residential,None,None
3,NaN,NaN,261.0,2.0,None,NaN,NaN,10711910,1878109,NaN,...,6.037113e+13,-0.041238,2017-03-17,None,None,None,Central,Single Family Residential,None,None
4,NaN,NaN,261.0,2.0,None,NaN,NaN,10711923,2190858,NaN,...,6.037113e+13,-0.009496,2017-03-24,None,None,None,Central,Single Family Residential,None,None


In [14]:
# Display the feature names, # of non-null values, and dtype.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52439 entries, 0 to 52438
Data columns (total 68 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   typeconstructiontypeid        76 non-null     float64
 1   storytypeid                   47 non-null     float64
 2   propertylandusetypeid         52439 non-null  float64
 3   heatingorsystemtypeid         33935 non-null  float64
 4   buildingclasstypeid           0 non-null      object 
 5   architecturalstyletypeid      70 non-null     float64
 6   airconditioningtypeid         13638 non-null  float64
 7   parcelid                      52439 non-null  int64  
 8   id                            52439 non-null  int64  
 9   basementsqft                  47 non-null     float64
 10  bathroomcnt                   52439 non-null  float64
 11  bedroomcnt                    52439 non-null  float64
 12  buildingqualitytypeid         33740 non-null  float64
 13  c

## Create a function to reproduce acquisition

Now that we've successfully imported the data, let's create a function to make data acquisition reporoducable.
- Combine get_connection() and sql variable to acquire Zillow data.

In [21]:
import os

def get_zillow_data():
    '''
    This function acquires the zillow dataset from a SQL database using Pandas and SQL.
    Returns the zillow dataset as a Pandas DataFrame.
    
    If the zillow data is stored in the current repository, it returns a Pandas DataFrame.
    If the zillow data does not exist in the current directory, it queries a SQL database for the dataset.
    A local copy will be created as a csv file in the current directory for future use.
    '''
    
    sql_query ='''
    select *
    from properties_2017
    join(select parcelid,
        logerror,
        max(transactiondate) as lasttransactiondate
        from predictions_2017
        group by parcelid, logerror
        ) as predictions using(parcelid)
    left join `airconditioningtype` using(`airconditioningtypeid`)
    left join `architecturalstyletype` using(`architecturalstyletypeid`)
    left join `buildingclasstype` using(`buildingclasstypeid`)
    left join `heatingorsystemtype` using(`heatingorsystemtypeid`)
    left join `propertylandusetype` using(`propertylandusetypeid`)
    left join `storytype` using(`storytypeid`)
    left join `typeconstructiontype` using(`typeconstructiontypeid`)
    where (latitude is not null
    and longitude is not null
    and propertylandusetypeid = 261);'''
    
    file = 'zillow.csv'
    
    if os.path.isfile(file):
        return pd.read_csv('zillow.csv')
    else:
        df = pd.read_sql(sql_query, get_connection('zillow'))
        df.to_csv('zillow.csv', index=False)
        return df

# Step 2: Store the dataset and load the local cached file

In [26]:
# Use the function from above to load cached data
df = get_zillow_data()

In [27]:
print(df.shape)
df.head()

(52439, 68)


,typeconstructiontypeid,storytypeid,propertylandusetypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,parcelid,id,basementsqft,...,censustractandblock,logerror,lasttransactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
0,NaN,NaN,261.0,2.0,NaN,NaN,NaN,10711855,1087254,NaN,...,6.037113e+13,-0.007357,2017-07-07,NaN,NaN,NaN,Central,Single Family Residential,NaN,NaN
1,NaN,NaN,261.0,2.0,NaN,NaN,1.0,10711877,1072280,NaN,...,6.037113e+13,0.021066,2017-08-29,Central,NaN,NaN,Central,Single Family Residential,NaN,NaN
2,NaN,NaN,261.0,2.0,NaN,NaN,1.0,10711888,1340933,NaN,...,6.037113e+13,0.077174,2017-04-04,Central,NaN,NaN,Central,Single Family Residential,NaN,NaN
3,NaN,NaN,261.0,2.0,NaN,NaN,NaN,10711910,1878109,NaN,...,6.037113e+13,-0.041238,2017-03-17,NaN,NaN,NaN,Central,Single Family Residential,NaN,NaN
4,NaN,NaN,261.0,2.0,NaN,NaN,NaN,10711923,2190858,NaN,...,6.037113e+13,-0.009496,2017-03-24,NaN,NaN,NaN,Central,Single Family Residential,NaN,NaN


In [29]:
# We've acquired properties that were defined as 'Single Family Residential'
df.propertylandusedesc.value_counts()

Single Family Residential    52439
Name: propertylandusedesc, dtype: int64

Now that we've acquired the data and saved it locally, let's move on to [data preparation](https://github.com/Promeos/zillow-clustering-project/blob/main/exploration.ipynb).

# Appendix

## SQL Query
Let's break down the query
```sql
-- First two lines establish the primary table to join on.
select *
from properties_2017

-- This subquery selects parcelid, logerror, and the last transaction date a property was sold in 2017. If a property was sold multiple times, we want the most recent date the property was sold. If a property was sold only one time, that's okay too. max(transactiondate) combined with a group by statement will retrieve the most recent transaction dates for a given parcelid. We can join this data on `properties_2017` using the `parcelid` column. 
join(select parcelid,
    logerror,
    max(transactiondate) as lasttransactiondate
    from predictions_2017
    group by parcelid, logerror
    ) as predictions using(parcelid)


-- As discussed above using the left/right join diagram, we'll join the rest of the tables using a left join to preserve the columns/rows in `properties_2017`
left join `airconditioningtype` using(`airconditioningtypeid`)
left join `architecturalstyletype` using(`architecturalstyletypeid`)
left join `buildingclasstype` using(`buildingclasstypeid`)
left join `heatingorsystemtype` using(`heatingorsystemtypeid`)
left join `propertylandusetype` using(`propertylandusetypeid`)
left join `storytype` using(`storytypeid`)
left join `typeconstructiontype` using(`typeconstructiontypeid`)

-- Use a where clause to filter for properties that have non-null latitude and longitude values.
where (latitude is not null
and longitude is not null
-- Filter for properties labeled as single unit / single family homes
and propertylandusetypeid = 261);
-- 261 == Single Family Residential

-- Include after MVP
-- 262 == Rural Residence
-- 264 == Townhouse
-- 273 == Bungalow
-- Filter for single story properties
-- 246 == Duplex (2 units any combination)
-- 247 == Triplex (3 units any combination)
-- 248 == Quadruplex (4 units any combination)
```

[Return to Loading data from the database](#_3._Load_data_from_the_database_to_our_local_environment)